In [1]:
# 1404-05-18, Mohammad Kadkhodaei
# -------------------------------

NUM_FOG_NODES = 7
NUM_EDGE_NODES = 14
NUM_CLOUD_NODES = 1

# Infrastructure parameters
CLOUD_vCPU = float('inf')  # Elastic
CLOUD_MEM = float('inf')   # Elastic
FOG_vCPU = 4
FOG_MEM = 4  # GB
EDGE_vCPU = 1
EDGE_MEM = 1  # GB


class Infrastructure:
    def __init__(self):
    
    
        self.cloud_nodes = [{'remaining_vcpu': CLOUD_vCPU, 'remaining_mem': CLOUD_MEM} for _ in range(NUM_CLOUD_NODES)]
        # print(self.cloud_nodes)
        self.fog_nodes = [{'remaining_vcpu': FOG_vCPU, 'remaining_mem': FOG_MEM} for _ in range(NUM_FOG_NODES)]
        # print(self.fog_nodes)
        self.edge_nodes = [{'remaining_vcpu': EDGE_vCPU, 'remaining_mem': EDGE_MEM} for _ in range(NUM_EDGE_NODES)]
    
    def get_feature_vector(self):
        """Returns the current state of infrastructure resources"""
        cloud_vcpu = [n['remaining_vcpu'] for n in self.cloud_nodes]
        cloud_mem = [n['remaining_mem'] for n in self.cloud_nodes]
        fog_vcpu = [n['remaining_vcpu'] for n in self.fog_nodes]
        fog_mem = [n['remaining_mem'] for n in self.fog_nodes]
        edge_vcpu = [n['remaining_vcpu'] for n in self.edge_nodes]
        edge_mem = [n['remaining_mem'] for n in self.edge_nodes]
        
        return cloud_vcpu + cloud_mem + fog_vcpu + fog_mem + edge_vcpu + edge_mem  

    def get_node_resources(self, node_type, node_idx):
        """Get remaining resources for a specific node"""
        if node_type == 'cloud':
            return self.cloud_nodes[node_idx]['remaining_vcpu'], self.cloud_nodes[node_idx]['remaining_mem']
        elif node_type == 'fog':
            return self.fog_nodes[node_idx]['remaining_vcpu'], self.fog_nodes[node_idx]['remaining_mem']
        else:  # edge
            return self.edge_nodes[node_idx]['remaining_vcpu'], self.edge_nodes[node_idx]['remaining_mem']

    def allocate_resources(self, node_type, node_idx, vcpu, mem):
        """Allocate resources to a node"""
        if node_type == 'cloud':
            # Cloud has elastic resources, no need to update
            pass
        elif node_type == 'fog':
            self.fog_nodes[node_idx]['remaining_vcpu'] -= vcpu
            self.fog_nodes[node_idx]['remaining_mem'] -= mem
        else:  # edge
            self.edge_nodes[node_idx]['remaining_vcpu'] -= vcpu
            self.edge_nodes[node_idx]['remaining_mem'] -= mem
            
    def get_data_transfer_cost(self, src_type, src_idx, dst_type, dst_idx):
        """Get data transfer cost between two nodes"""
        if src_type == 'cloud':
            return TYPE1_COST
        elif src_type == 'fog':
            if dst_type == 'cloud':
                return TYPE2_COST
            elif dst_type == 'fog':
                if dst_idx in self.fog_edge_mapping.get(src_idx, []):
                    return TYPE3_COST
                return TYPE2_COST
            else:  # edge
                if dst_idx in self.fog_edge_mapping.get(src_idx, []):
                    return TYPE3_COST
                return TYPE2_COST
        else:  # edge
            if dst_type == 'cloud':
                return TYPE2_COST
            elif dst_type == 'fog':
                if src_idx in self.fog_edge_mapping.get(dst_idx, []):
                    return TYPE3_COST
                return TYPE2_COST
            else:  # edge
                if src_idx == dst_idx:
                    return 0  # same node
                return TYPE2_COST
            
    def get_latency(self, node_type):
        """Get latency for a node type"""
        if node_type == 'cloud':
            return 5000  # ms
        else:
            return 0  # fog and edge have negligible latency

In [2]:
infra = Infrastructure()
print(infra.get_feature_vector())
print('---')
print(type(infra.get_node_resources('edge', 13)))

[inf, inf, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
---
<class 'tuple'>
